# 5.4 Aplicaciones

## 5.4.1 Aplicaciones del polinomio característico

El **polinomio característico** no es únicamente una herramienta algebraica para calcular valores propios; en realidad, constituye un puente fundamental entre el álgebra lineal y múltiples áreas aplicadas. A través de sus raíces (los valores propios), el polinomio característico permite comprender la **dinámica, estabilidad y estructura** de sistemas complejos modelados mediante matrices.

A continuación se presentan algunas de sus aplicaciones más relevantes.

### Cadenas de Markov

Las **cadenas de Markov** modelan sistemas estocásticos que evolucionan en el tiempo, donde el estado futuro depende únicamente del estado actual. Estas cadenas se representan mediante una **matriz de transición** $P$, cuyas entradas cumplen:

$$
P_{ij} \ge 0, \quad \sum_{j} P_{ij} = 1.
$$

El comportamiento a largo plazo de la cadena está determinado por los **valores propios de $P$**, los cuales se obtienen como raíces de su polinomio característico:

$$
p_P(\lambda) = \det(P - \lambda I).
$$

Algunas propiedades clave son:

- El valor propio $\lambda = 1$ siempre existe. **(¿Por qué?)**
- Si $\lambda = 1$ tiene multiplicidad algebraica 1 y el resto de los valores propios satisfacen $|\lambda| < 1$, la cadena converge a una **distribución estacionaria única**.
- La rapidez de convergencia depende de la magnitud del segundo valor propio más grande en valor absoluto.

Así, el polinomio característico permite estudiar:
- estabilidad del sistema,
- existencia de equilibrio,
- velocidad de mezcla de la cadena.

Estas ideas son fundamentales en:
- modelos económicos,
- algoritmos de ranking (como PageRank),
- simulaciones estocásticas.



#### Ejemplo numérico: cadenas de Markov en el contexto de PageRank

Imaginemos 3 páginas web: $A$, $B$ y $C$, con los siguientes enlaces:

- $A \to B$, $A \to C$
- $B \to C$
- $C \to A$

Si usamos una matriz de transición por “seguir un enlace” (sin teletransportación), cada página reparte probabilidad uniforme entre sus enlaces salientes. En el orden $(A,B,C)$, la matriz de transición $P$ queda:

$$
P=
\begin{pmatrix}
0 & \tfrac12 & \tfrac12 \\
0 & 0 & 1 \\
1 & 0 & 0
\end{pmatrix}.
$$

En PageRank se suele usar **factor de amortiguamiento** (teletransportación) $d\in(0,1)$ (típicamente $d=0.85$). Con teletransportación uniforme, la matriz final es:

$$
G = dP + (1-d)\tfrac{1}{n}\mathbf{1}\mathbf{1}^\top,
\quad n=3.
$$

Con $d=0.85$ se tiene $(1-d)/3 = 0.05$, por lo que

$$
G=
\begin{pmatrix}
0.05 & 0.475 & 0.475 \\
0.05 & 0.05  & 0.90 \\
0.90 & 0.05  & 0.05
\end{pmatrix}.
$$

El **PageRank** es la distribución estacionaria $\pi$ (vector fila) que satisface:

$$
\pi = \pi G,
\qquad
\pi_A+\pi_B+\pi_C=1.
$$

Resolviendo el sistema (equivalentemente, buscando el vector propio asociado a $\lambda=1$), se obtiene aproximadamente:

$$
\pi \approx (0.388,\ 0.215,\ 0.397).
$$

Interpretación: en este mini-web, $C$ y $A$ quedan con ranking similar (ligeramente mayor $C$), y $B$ más abajo.


#### ¿Cómo hacerlo con Python?




In [1]:
# Ejemplo 1
import numpy as np

# Matriz P (A,B,C)
P = np.array([
    [0.0, 0.5, 0.5],
    [0.0, 0.0, 1.0],
    [1.0, 0.0, 0.0]
])

d = 0.85
n = P.shape[0]
G = d * P + (1 - d) * np.ones((n, n)) / n

# Power iteration
pi = np.ones(n) / n  # distribución inicial
for _ in range(200):
    pi_next = pi @ G
    if np.linalg.norm(pi_next - pi, 1) < 1e-12:
        break
    pi = pi_next

print("G=\n", G)
print("PageRank pi =", pi, "  suma=", pi.sum())

G=
 [[0.05  0.475 0.475]
 [0.05  0.05  0.9  ]
 [0.9   0.05  0.05 ]]
PageRank pi = [0.38778971 0.21481063 0.39739966]   suma= 1.0000000000000002


In [4]:
#Ejemplo 2
import numpy as np

# (reusa G de arriba)
w, V = np.linalg.eig(G.T)  # eigen de G^T

# índice del autovalor más cercano a 1
idx = np.argmin(np.abs(w - 1))
pi = np.real(V[:, idx])
pi = pi / pi.sum()         # normalizar para que sume 1
pi = np.maximum(pi, 0)     # por estabilidad numérica

print("Autovalores:", w)
print("PageRank pi =", pi, "  suma=", pi.sum())


Autovalores: [ 1.   +0.j    -0.425+0.425j -0.425-0.425j]
PageRank pi = [0.38778971 0.21481063 0.39739966]   suma= 1.0




### Análisis de grafos y redes

En teoría de grafos, el polinomio característico aparece de forma natural al estudiar el **espectro** de matrices asociadas a un grafo, como la **matriz de adyacencia** $A$ o la **matriz laplaciana** $L$.

Para un grafo con $n$ nodos, el polinomio característico se define como:

$$
p_A(\lambda) = \det(A - \lambda I), \quad
p_L(\lambda) = \det(L - \lambda I).
$$

Las raíces de estos polinomios proporcionan información estructural profunda:

- conectividad global del grafo,
- número de componentes conexas,
- robustez frente a fallas,
- patrones de difusión de información.

En particular:
- La multiplicidad del valor propio $\lambda = 0$ del laplaciano indica el número de componentes conexas.
- Los valores propios dominantes de la matriz de adyacencia reflejan la presencia de comunidades densamente conectadas.

El polinomio característico permite entonces **clasificar grafos**, comparar redes y estudiar su comportamiento dinámico, con aplicaciones en:
- redes sociales,
- redes de transporte,
- redes biológicas y neuronales.

---

### Ciencia de datos: Análisis de Componentes Principales (PCA)

En **ciencia de datos**, el polinomio característico juega un papel central en el **Análisis de Componentes Principales (PCA)**, una técnica fundamental para reducción de dimensionalidad.

Dado un conjunto de datos, se construye la **matriz de covarianza** $C$, que es simétrica. Los valores propios de $C$ se obtienen resolviendo:

$$
p_C(\lambda) = \det(C - \lambda I).
$$

Cada valor propio mide la **varianza explicada** por su correspondiente componente principal, mientras que los vectores propios determinan las direcciones de máxima variabilidad en los datos.

Gracias al polinomio característico, el PCA permite:
- identificar variables redundantes,
- reducir dimensionalidad sin perder información relevante,
- visualizar datos de alta dimensión,
- mejorar modelos de aprendizaje automático.

Esta técnica es ampliamente utilizada en:
- análisis exploratorio de datos,
- visión por computadora,
- procesamiento de señales,
- bioinformática.

---

### Conclusión

El polinomio característico es una herramienta central que conecta el álgebra lineal con aplicaciones modernas en probabilidad, redes complejas y ciencia de datos. A través del estudio de sus raíces, es posible comprender tanto la estructura como la dinámica de sistemas reales, lo que explica su importancia en matemáticas aplicadas, ingeniería y ciencias computacionales.

## 5.4.2 Aplicaciones de la diagonalización de matrices

La diagonalización de matrices es una de las herramientas más importantes del álgebra lineal, ya que permite transformar una matriz compleja en una forma simple sin perder información esencial. Cuando una matriz $A$ es diagonalizable, su comportamiento puede analizarse de manera clara a través de sus valores propios y vectores propios, lo cual tiene numerosas aplicaciones en matemáticas, ciencia y tecnología.



### Cálculo de potencias de matrices

Una de las aplicaciones más directas de la diagonalización es el cálculo eficiente de potencias de matrices. Si una matriz $A$ es diagonalizable, es decir,

$$
A = P D P^{-1},
$$

entonces

$$
A^k = P D^k P^{-1},
$$

para cualquier entero $k \geq 1$.

Como $D$ es diagonal, calcular $D^k$ es trivial, ya que basta elevar cada valor propio a la potencia $k$. Esta propiedad es fundamental en:
- modelos de crecimiento poblacional,
- cadenas de Markov,
- sistemas dinámicos discretos.

**¿Por qué?



### Análisis de grafos y redes

En teoría de grafos, la diagonalización de matrices como la **matriz de adyacencia** o la **matriz laplaciana** permite estudiar propiedades estructurales de redes complejas.

Los valores propios proporcionan información sobre:
- conectividad del grafo,
- número de componentes conexas,
- propagación de información o fallas,
- detección de comunidades.

Este enfoque es ampliamente utilizado en el análisis de redes sociales, redes de transporte y redes biológicas.

---

### Ciencia de datos y reducción de dimensionalidad

La diagonalización es la base matemática de técnicas fundamentales en ciencia de datos, como el **Análisis de Componentes Principales (PCA)**. En PCA, se diagonaliza la matriz de covarianza de los datos para identificar las direcciones de máxima variabilidad.

Este procedimiento permite:
- reducir la dimensión de los datos,
- eliminar redundancia,
- visualizar datos de alta dimensión,
- mejorar el rendimiento de modelos de aprendizaje automático.

---




### Resumen

La diagonalización de matrices permite:
- simplificar cálculos complejos,
- comprender la estructura interna de una transformación lineal,
- analizar estabilidad y comportamiento a largo plazo,
- desarrollar modelos eficientes en ciencia e ingeniería.

Por estas razones, la diagonalización es una herramienta central en el álgebra lineal moderna y una pieza clave en numerosas aplicaciones contemporáneas.
